In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from typing import Any # import Any from the typing module

# URL of the website
URL = "https://www.redbus.in/online-booking/meghalaya-transport-corporation-mtc"

def initialize_driver():
    # Initialize the Chrome driver
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    # Load the page
    driver.get(url)
    time.sleep(5) # wait for the page to load

# Functions to scrape bus routes
def scrape_bus_routes(driver):
    # Scrape bus routes
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Functions to scrape bus details
def scrape_bus_details(driver, url, route_name):
    # Scrape bus details
    try:
        driver.get(url)
        time.sleep(5) # wait for some 5 sec for the page to load

        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5) # wait for some 5 sec for the page to load

            #scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5) # wait for some 5 sec for the page to load

            # Find but details from the availbility
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPATH to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text.strip(),
                    "Bus_Type": bus_type_elements[i].text.strip(),
                    "Departing_Time": departing_time_elements[i].text.strip(),
                    "Duration": duration_elements[i].text.strip(),
                    "Reaching_Time": reaching_time_elements[i].text.strip(),
                    "Star_Rating": star_rating_elements[i].text.strip() if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text.strip(),
                    "Seat_Availability": seat_availability_elements[i].text.strip() if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details

        except Exception as e:
           print(f"Error occurred while scrapping bus details for {url}: {str(e)}")
           return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []
# List to hold all bus details
all_bus_details = []

# Main function to execute the scraping process
def scrap_all_pages():
    for page in range(1, 3): # There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5) # wait for some 5 sec for the page to load
                
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            # Iterate over each bus route link and scrape details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                  all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrap_all_pages()

# Create a DataFrame from the scraped data
df_buses_7 = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df_buses_7.to_csv('Meghalaya_bus_details.csv', index=False)

In [2]:
df_buses_7

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Shillong to Tura (Meghalaya),https://www.redbus.in/bus-tickets/shillong-to-...,Meghalaya Transport Corporation(MTC) - 163306,Tata Non A/C Seater Executive Luxury (2+2),17:00,10h 30m,03:30,3.3,INR 517,6 Seats available
1,Shillong to Tura (Meghalaya),https://www.redbus.in/bus-tickets/shillong-to-...,NETWORK TRAVELS,NON A/C Seater (2+2),16:30,13h 30m,06:00,3.7,650,36 Seats available
2,Guwahati to Shillong,https://www.redbus.in/bus-tickets/guwahati-to-...,Meghalaya Transport Corporation(MTC) - 163300,Force Motors Non A/C Seater Executive (2+2),07:00,03h 00m,10:00,4.3,INR 200,4 Seats available
3,Guwahati to Shillong,https://www.redbus.in/bus-tickets/guwahati-to-...,Meghalaya Transport Corporation(MTC) - 175777,Force Motors Non A/C Seater Executive (2+2),12:00,03h 00m,15:00,4.4,INR 200,11 Seats available
4,Guwahati to Shillong,https://www.redbus.in/bus-tickets/guwahati-to-...,NETWORK TRAVELS,Bharat Benz A/C Sleeper (2+1),05:10,02h 30m,07:40,3.2,INR 400,26 Seats available
5,Guwahati to Shillong,https://www.redbus.in/bus-tickets/guwahati-to-...,Upasana Travels (Under ASTC),Non AC Seater (2+2),11:00,03h 30m,14:30,3.0,INR 300,19 Seats available
6,Tura (Meghalaya) to Shillong,https://www.redbus.in/bus-tickets/tura-meghala...,Meghalaya Transport Corporation(MTC) - 163370,Tata Non A/C Seater Executive Luxury (2+2),18:30,09h 30m,04:00,3.3,INR 517,5 Seats available
7,Tura (Meghalaya) to Shillong,https://www.redbus.in/bus-tickets/tura-meghala...,NETWORK TRAVELS,NON A/C Seater (2+2),19:00,13h 00m,08:00,3.2,650,24 Seats available
8,Shillong to Guwahati,https://www.redbus.in/bus-tickets/shillong-to-...,Meghalaya Transport Corporation(MTC) - 169753,Force Motors Non A/C Seater Executive (2+2),07:00,02h 30m,09:30,4.5,INR 200,20 Seats available
9,Shillong to Guwahati,https://www.redbus.in/bus-tickets/shillong-to-...,Meghalaya Transport Corporation(MTC) - 172215,Force Motors Non A/C Seater Executive (2+2),15:00,03h 00m,18:00,4.4,INR 200,21 Seats available
